In [ ]:
!pip install tensorflow-recommenders

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from itertools import combinations

In [ ]:
df = pd.read_csv('data_wisata.csv')
df

,Nama Wisata,Deskripsi,Lokasi / Tempat,Fasilitas Yang Tersedia,Aksesibilitas,Jenis Wisata,Kategori,Child Friendly,Alamat,Link Alamat,Harga,Rentang Harga,Rating,latitude,longitude
0,Pantai Licin,"Pantai Licin terletak di Desa Lebakharjo, Keca...","Desa Lebakharjo, Ampelgading","Tempat parkir, Warung makan, Toilet, Mushola, ...","makadam, kendaraan roda 2",Pantai,"Family, Solo",Yes,"Area Sawah/Kebun, Lebakharjo, Ampelgading, Kab...",https://www.google.com/maps/place/Pantai+Licin...,0,0,4.3,-8.363503,112.946779
1,Pantai Taman Ayu,Pantai Taman Ayu berlokasi di Desa Sumberbenin...,"Desa Bandungrejo, Bantur","Tempat parkir, Warung makan, Toilet, Mushola, ...","makadam, kendaraan roda 2 dan 4",Pantai,"Family, Solo",Yes,"Pantai taman ayu, Area Gn., Sidurejo, Gedangan...",https://www.google.com/maps/place/Pantai+Taman...,20000,3,4.1,-8.407483,112.573002
2,Pantai Balekambang,"Pantai Balekambang terletak di Desa Srigonco, ...","Desa Srigonco, Bantur","Tempat parkir, Warung makan, Toilet, Mushola, ...","aspal, kendaraan roda 2 dan 4, bus",Pantai,"Family, Solo",Yes,"Jalan Balekambang, Dusun Sumber Jambe, Desa Sr...",https://www.google.com/maps/place/Balekambang+...,20000,3,4.5,-8.403571,112.539371
3,Pantai Regent,"Pantai Regent berada di Desa Tambakrejo, Kecam...","Desa Srigonco, Bantur","Tempat parkir, Warung makan, Toilet, Mushola, ...","aspal, kendaraan roda 2 dan 4, bus",Pantai,"Family, Solo",Yes,"Jl. Balekambang, Balaikambang, Srigonco, Kec. ...",https://www.google.com/maps/place/Pantai+Regen...,15000,2,4.5,-8.403426,112.545205
4,Pantai Kondang Merak,Pantai Kondang Merak terletak di Desa Sumberbe...,"Desa Sumberbening, Bantur","Tempat parkir, Warung makan, Toilet, Mushola, ...","makadam, kendaraan roda 2 dan 4",Pantai,"Family, Solo",Yes,"Desa Sumberbening, Kecamatan Bantur, Kabupaten...",https://www.google.com/maps/place/Kondang+Mera...,10000,2,4.4,-8.396575,112.519074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Taman Bunga Selecta,Taman Bunga Selecta adalah taman rekreasi yang...,Tulungrejo,"Kebun bunga, Greenhouse, Perahu wisata, Area b...",aspal. Kendaraan roda 2 dan 13,Taman,"Friends, Solo",No,"Jl. Raya Selecta No.mor 1, Tulungrejo, Kec. Bu...",https://www.google.com/maps/place/Taman+Bunga+...,40000,5,4.6,-7.822086,112.526612
136,Taman Hutan Kota Batu,Taman Hutan Kota Batu adalah area hijau di ten...,Sisir,"Area jogging, Arena bermain, Taman satwa, Area...",aspal. Kendaraan roda 2 dan 14,Taman,"Friends, Solo",No,"Jl. Sultan Agung, Sisir, Kec. Batu, Kota Batu,...",https://www.google.com/maps/place/Taman+Hutan+...,0,0,4.5,-7.879386,112.516195
137,Taman Eco Green,Taman Eco Green adalah destinasi wisata yang m...,Sisir,"Pemandian air panas, Kolam renang, Outbound, A...",aspal. Kendaraan roda 2 dan 15,Taman,"Friends, Solo",No,"Jl. Raya Oro-Oro Ombo No.9A, Sisir, Kec. Batu,...",https://www.google.com/maps/place/Eco+Green+Pa...,20000,3,4.6,-7.879386,112.516195
138,Gunung Seruk,Gunung Seruk merupakan destinasi pendakian yan...,Batu,"Jalur hiking, Air terjun, Goa Jepang",aspal. Kendaraan roda 2 dan 16,Gunung,Solo,No,"Kehutanan, Kec. Batu, Kota Batu, Jawa Timur",https://www.google.com/maps/place/Gunung+Seruk...,5000,1,5.0,-7.881667,112.434962


In [ ]:
df = df.drop(columns=['Rentang Harga', 'Lokasi / Tempat'], axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Nama Wisata              140 non-null    object 
 1   Deskripsi                140 non-null    object 
 2   Fasilitas Yang Tersedia  140 non-null    object 
 3   Aksesibilitas            140 non-null    object 
 4   Jenis Wisata             140 non-null    object 
 5   Kategori                 140 non-null    object 
 6   Child Friendly           140 non-null    object 
 7   Alamat                   140 non-null    object 
 8   Link Alamat              140 non-null    object 
 9   Harga                    140 non-null    int64  
 10  Rating                   140 non-null    float64
 11  latitude                 140 non-null    float64
 12  longitude                140 non-null    float64
dtypes: float64(3), int64(1), object(9)
memory usage: 14.3+ KB


In [ ]:
# Convert latitude and longitude to numeric, coercing errors to NaN
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Drop rows with NaN values in latitude and longitude
df = df.dropna(subset=['latitude', 'longitude'])

In [ ]:
# Convert harga to numeric
df['Harga'] = pd.to_numeric(df['Harga'], errors='coerce')

In [ ]:
def filter_data(df, user_kategori, user_jenis_wisata, user_child_friendly):
    filter_kategori = df[df['Kategori'].str.contains(user_kategori, case=False, na=False)]
    filter_jenis = filter_kategori[filter_kategori['Jenis Wisata'].str.contains('|'.join(user_jenis_wisata), case=False, na=False)]
    if user_child_friendly.lower() == 'yes':
        filter_child = filter_jenis[filter_jenis['Child Friendly'].str.contains('Yes', case=False, na=False)]
    else:
        filter_child = filter_jenis

    # Tambahkan kondisi jika tidak ada data yang sesuai setelah filtering
    if filter_child.empty:
        filter_child = df[df['Jenis Wisata'].str.contains('|'.join(user_jenis_wisata), case=False, na=False)]

    return filter_child

# Menambahkan kolom "Koordinat" sebagai tuple dari Latitude dan Longitude
df['Koordinat'] = list(zip(df['latitude'], df['longitude']))

# Definisikan model rekomendasi
class TripModel(tfrs.Model):

    def __init__(self):
        super().__init__()

        self.item_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=df["Jenis Wisata"].unique(), mask_token=None),
            tf.keras.layers.Embedding(len(df["Jenis Wisata"].unique()) + 1, 32)
        ])

        self.description_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=df["Deskripsi"].unique(), mask_token=None),
            tf.keras.layers.Embedding(len(df["Deskripsi"].unique()) + 1, 32)
        ])

        self.facilities_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=df["Fasilitas Yang Tersedia"].unique(), mask_token=None),
            tf.keras.layers.Embedding(len(df["Fasilitas Yang Tersedia"].unique()) + 1, 32)
        ])

        self.accessibility_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=df["Aksesibilitas"].unique(), mask_token=None),
            tf.keras.layers.Embedding(len(df["Aksesibilitas"].unique()) + 1, 32)
        ])

        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(1)
        ])

        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, features, training=False):
        item_embeddings = self.item_embedding(features["Jenis Wisata"])
        description_embeddings = self.description_embedding(features["Deskripsi"])
        facilities_embeddings = self.facilities_embedding(features["Fasilitas Yang Tersedia"])
        accessibility_embeddings = self.accessibility_embedding(features["Aksesibilitas"])

        embeddings = tf.concat([item_embeddings, description_embeddings, facilities_embeddings, accessibility_embeddings], axis=1)

        ratings = self.rating_model(embeddings)

        return self.task(
            labels=features["Rating"],
            predictions=ratings
        )

model = TripModel()

# Compile and train the model
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Asumsikan kita sudah memiliki data yang diformat dengan benar
features = {
    "Jenis Wisata": df["Jenis Wisata"].values,
    "Deskripsi": df["Deskripsi"].values,
    "Fasilitas Yang Tersedia": df["Fasilitas Yang Tersedia"].values,
    "Aksesibilitas": df["Aksesibilitas"].values,
    "Rating": df["Rating"].values
}

# Train model
model.fit(features, epochs=10)
model.save('model.tf')


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 6.9199 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 3.4891 - total_loss: 6.9199  
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.4339 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 1.2701 - total_loss: 1.4339 
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6185 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 0.7916 - total_loss: 0.6185 
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3284 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 0.6166 - total_loss: 0.3284  
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2348 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 0.5022 - total_loss: 0.2348 
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0750 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 0.2804 - total_loss: 0.0750 
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0498 - regularization_

In [ ]:
def filter_and_recommend(df, user_kategori, user_jenis_wisata, user_child_friendly, user_budget, user_latitude, user_longitude, num_recommendations):
    filtered_data = filter_data(df, user_kategori, user_jenis_wisata, user_child_friendly)

    # Menghitung jarak antara koordinat user dan koordinat wisata
    def calculate_distance(coord1, coord2):
        return ((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)**0.5

    user_coords = (user_latitude, user_longitude)
    filtered_data["Distance"] = filtered_data["Koordinat"].apply(lambda x: calculate_distance(user_coords, x))

    # Menggunakan model untuk memberikan rating prediksi
    features_for_prediction = {
        "Jenis Wisata": filtered_data["Jenis Wisata"].values,
        "Deskripsi": filtered_data["Deskripsi"].values,
        "Fasilitas Yang Tersedia": filtered_data["Fasilitas Yang Tersedia"].values,
        "Aksesibilitas": filtered_data["Aksesibilitas"].values
    }

    item_embeddings = model.item_embedding(features_for_prediction["Jenis Wisata"])
    description_embeddings = model.description_embedding(features_for_prediction["Deskripsi"])
    facilities_embeddings = model.facilities_embedding(features_for_prediction["Fasilitas Yang Tersedia"])
    accessibility_embeddings = model.accessibility_embedding(features_for_prediction["Aksesibilitas"])

    embeddings = tf.concat([item_embeddings, description_embeddings, facilities_embeddings, accessibility_embeddings], axis=1)
    if filtered_data.empty:
      predicted_ratings = model.rating_model(embeddings).numpy().flatten()
      filter_kategori = df[df['Kategori'].str.contains(user_kategori, case=False, na=False)]
      filter_kategori["Predicted Rating"] = predicted_ratings
      recommendations = filter_kategori[filter_kategori['Harga'] <= user_budget]
    else:
      predicted_ratings = model.rating_model(embeddings).numpy().flatten()
      filtered_data["Predicted Rating"] = predicted_ratings
      # Filter berdasarkan budget
      recommendations = filtered_data[filtered_data['Harga'] <= user_budget]

    if recommendations.empty:
        print("Tidak ada rekomendasi dalam budget yang diberikan.")
        recommendations = filtered_data.sort_values(by='Harga').head(num_recommendations)
        return recommendations

    # Urutkan rekomendasi berdasarkan jarak terdekat ke user
    recommendations["Distance"] = recommendations["Koordinat"].apply(lambda x: calculate_distance(user_coords, x))
    recommendations = recommendations.sort_values(by="Distance")
    # Reset index to ensure we don't get KeyError
    recommendations = recommendations.reset_index(drop=True)

    best_combination = None
    best_rating_sum = -np.inf
    best_distance_sum = np.inf
    combination_count = 0
    max_combinations = 1000

    # Coba semua kombinasi dari 1 hingga num_recommendations tempat wisata
    for r in range(1, num_recommendations + 1):
        if combination_count > max_combinations:
            break
        for combination in combinations(recommendations.index, r):
            combination_count += 1
            if combination_count > max_combinations:
                break
            total_cost = recommendations.loc[list(combination), 'Harga'].sum()
            if total_cost <= user_budget:
                rating_sum = recommendations.loc[list(combination), 'Predicted Rating'].sum()
                distance_sum = recommendations.loc[list(combination), 'Distance'].sum()
                if rating_sum > best_rating_sum or (rating_sum == best_rating_sum and distance_sum < best_distance_sum):
                    best_rating_sum = rating_sum
                    best_distance_sum = distance_sum
                    best_combination = combination

    if best_combination:
        return recommendations.loc[list(best_combination)].sort_values(by='Predicted Rating', ascending=False)
    else:
        print("Tidak ada tempat wisata yang sesuai dengan preferensi yang diberikan.")
        return pd.DataFrame()

In [ ]:
user_kategori = "Family"
user_jenis_wisata = ["Pantai"]
user_child_friendly = "No"
user_budget = 0
user_latitude = 112.752088
user_longitude = -7.257472
num_recommendations = 5

recommended_places = filter_and_recommend(df, user_kategori, user_jenis_wisata, user_child_friendly, user_budget, user_latitude, user_longitude, num_recommendations)
print(recommended_places)

           Nama Wisata                                          Deskripsi  \
0  Pantai Kondang Iwak  Pantai Kondang Iwak terletak di Desa Sumberben...   
1    Pantai Ndaliputih  Pantai Ndaliputih berlokasi di Desa Tambakrejo...   
2   Pantai Jailangkung  Pantai Jailangkung berlokasi di Desa Tambakrej...   
3         Pantai Licin  Pantai Licin terletak di Desa Lebakharjo, Keca...   

                             Fasilitas Yang Tersedia  \
0  Tempat parkir, Warung makan, Toilet, Mushola, ...   
1  Tempat parkir, Warung makan, Toilet, Mushola, ...   
2  Tempat parkir, Warung makan, Toilet, Mushola, ...   
3  Tempat parkir, Warung makan, Toilet, Mushola, ...   

                     Aksesibilitas Jenis Wisata      Kategori Child Friendly  \
0  makadam, kendaraan roda 2 dan 4       Pantai  Family, Solo            Yes   
1  makadam, kendaraan roda 2 dan 4       Pantai  Family, Solo            Yes   
2  makadam, kendaraan roda 2 dan 4       Pantai  Family, Solo            Yes   
3        maka

<ipython-input-22-16494313926c>:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations["Distance"] = recommendations["Koordinat"].apply(lambda x: calculate_distance(user_coords, x))


In [ ]:
# Simpan model ke dalam format TensorFlow SavedModel
#tf.saved_model.save(model, 'trip_model_savedmodel')

In [ ]:
!pip install tensorflowjs

In [ ]:
import tensorflow as tf
import tensorflowjs as tfjs
import shutil
from google.colab import files
import zipfile
import os

# Load model dari TensorFlow
model = tf.keras.applications.MobileNetV2()

# Simpan model sebagai SavedModel
saved_model_dir = 'tripplan_model_savedmodel'
tf.saved_model.save(model, saved_model_dir)

# Konversi model ke TensorFlow.js format
output_tfjs_path = 'tfjs_model'
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model \
                        --signature_name=serving_default --saved_model_tags=serve \
                        {saved_model_dir} {output_tfjs_path}

# Fungsi untuk membuat file zip
def zip_dir(directory, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(directory, '..')))

# Kompres direktori tfjs_model menjadi zip
zip_filename = 'tfjs_model.zip'
zip_dir(output_tfjs_path, zip_filename)

# Unduh file zip ke perangkat lokal
files.download(zip_filename)


14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
2024-06-16 06:41:28.594489: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>